In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
import os
import re as re
import numpy as np
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from random import randint
from time import sleep

In [65]:
detroit_zip_codes_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/ZipCodes/Detriot_ZipCodes.txt'
num_files = len(os.listdir('C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/ZipCodes/HTML_Data/'))
detroit_zip_codes = read_zip_code_txt(detroit_zip_codes_path)
detroit_zip_codes = detroit_zip_codes[num_files:]

In [66]:
driver = init_driver("C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/ChromeDriver/chromedriver.exe")

C:\Users\austi\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [64]:
get_all_zipcode_data(zip_code_list=detroit_zip_codes, driver=driver)


CAPTCHA!
Manually complete the captcha requirements.
Once that's done, if the program was in the middle of scraping (and is still running), it should resume scraping after ~30 seconds.


KeyboardInterrupt: 

In [ ]:
close_connection(driver)

In [44]:
detroit_48126 = get_html(driver=driver)

In [46]:
x = BeautifulSoup(detroit_48126[0])

In [54]:
def read_zip_code_txt(file_path):
    file = open(file_path, 'r')
    for line in file:
        zipcodes = line.split(',')
    zipcodes = [x.strip(' ') for x in zipcodes]
    return zipcodes

def zillow_zip_code_links(zip_code_list):
    links = []
    base_link = 'https://www.zillow.com/homes/'
    for zip_code in zip_code_list:
        links.append(base_link + zip_code + '_rb/')
    return links

# Make sure to have driver open and activated before this is started
def get_all_zipcode_data(zip_code_list, driver):
    saved_dir = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/ZipCodes/HTML_Data/'
    #sleep(randint(10,13))
    #navigate_to_website(driver=driver, site='https://www.zillow.com')
    sleep(randint(10,50))
    
    for zip_code in zip_code_list:
        link = 'https://www.zillow.com/homes/' + zip_code +'_rb/'
        navigate_to_website(driver=driver, site=link)
        sleep(randint(1,10))
        zipcode_html = get_html(driver=driver)
        with open(saved_dir + zip_code + ".html", "w") as file:
            file.write(str(zipcode_html[0]))
        file.close()
        sleep(randint(1,14))
    close_connection(driver)

In [41]:
def init_driver(file_path):
    # Starting maximized fixes https://github.com/ChrisMuir/Zillow/issues/1
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(executable_path=file_path,
                              chrome_options=options)
    driver.wait = WebDriverWait(driver, 10)
    return(driver)

# Helper function for checking for the presence of a web element.
def _is_element_displayed(driver, elem_text, elem_type):
    if elem_type == "class":
        try:
            out = driver.find_element_by_class_name(elem_text).is_displayed()
        except (NoSuchElementException, TimeoutException):
            out = False
    elif elem_type == "css":
        try:
            out = driver.find_element_by_css_selector(elem_text).is_displayed()
        except (NoSuchElementException, TimeoutException):
            out = False
    else:
        raise ValueError("arg 'elem_type' must be either 'class' or 'css'")
    return(out)


# If captcha page is displayed, this function will run indefinitely until the
# captcha page is no longer displayed (checks for it every 30 seconds).
# Purpose of the function is to "pause" execution of the scraper until the
# user has manually completed the captcha requirements.
def _pause_for_captcha(driver):
    while True:
        time.sleep(30)
        if not _is_element_displayed(driver, "captcha-container", "class"):
            break

# Check to see if the page is currently stuck on a captcha page. If so, pause
# the scraper until user has manually completed the captcha requirements.
def check_for_captcha(driver):
    if _is_element_displayed(driver, "captcha-container", "class"):
        print("\nCAPTCHA!\n"\
              "Manually complete the captcha requirements.\n"\
              "Once that's done, if the program was in the middle of scraping "\
              "(and is still running), it should resume scraping after ~30 seconds.")
        _pause_for_captcha(driver)

def navigate_to_website(driver, site):
    driver.get(site)
    # Check to make sure a captcha page is not displayed.
    check_for_captcha(driver)

def _is_element_displayed(driver, elem_text, elem_type):
    if elem_type == "class":
        try:
            out = driver.find_element_by_class_name(elem_text).is_displayed()
        except (NoSuchElementException, TimeoutException):
            out = False
    elif elem_type == "css":
        try:
            out = driver.find_element_by_css_selector(elem_text).is_displayed()
        except (NoSuchElementException, TimeoutException):
            out = False
    else:
        raise ValueError("arg 'elem_type' must be either 'class' or 'css'")
    return(out)

def get_html(driver):
    output = []
    keep_going = True
    while keep_going:
        # Pull page HTML
        try:
            output.append(driver.page_source)
        except TimeoutException:
            pass
        # Check to see if a "next page" link exists.
        keep_going = _is_element_displayed(driver, "zsg-pagination-next",
                                           "class")
        if keep_going:
            # Test to ensure the "updating results" image isnt displayed.
            # Will try up to 5 times before giving up, with a 5 second wait
            # between each try.
            tries = 5
            cover = _is_element_displayed(driver,
                                          "list-loading-message-cover",
                                          "class")
            while cover and tries > 0:
                time.sleep(5)
                tries -= 1
                cover = _is_element_displayed(driver,
                                              "list-loading-message-cover",
                                              "class")
            # If the "updating results" image is confirmed to be gone
            # (cover == False), click next page. Otherwise, give up on trying
            # to click thru to the next page of house results, and return the
            # results that have been scraped up to the current page.
            if not cover:
                div = driver.find_element_by_class_name('zsg-pagination-next')
                try:
                    myElementList = [div.find_element_by_css_selector('a').get_attribute('href')]
                except NoSuchElementException:
                    keep_going = False
                else:
                    try:
                        driver.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "zsg-pagination-next"))).click()
                        time.sleep(3)
                        # Check to make sure a captcha page is not displayed.
                        check_for_captcha(driver)
                    except TimeoutException:
                        keep_going = False
            else:
                keep_going = False
    return(output)

# Teardown webdriver.
def close_connection(driver):
    driver.quit()